Ever wondered how you have fared in the Kaggle over time? 

This kernel is to look at the kaggle journey of you (and probably the people whom we silently stalk at) and understand more about the journey !

Please enter the `username` of the person in the below cell and execute the notebook to look at the journey.

In [4]:
import os
import string
import datetime
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

print(os.listdir("../input"))

['meta-kaggle']


In [5]:
### Enter the user name here ###
user_name = "kurianbenoy"

In [9]:
users_df = pd.read_csv("../input/meta-kaggle/Users.csv")
user_df = users_df[users_df["UserName"]==user_name]
user_id = user_df["Id"].values[0]
user_display = user_df["DisplayName"].values[0]
print("The user id for the given user name is : ",user_id)
print("The display name for the given user name is : ",user_display)

The user id for the given user name is :  1310697
The display name for the given user name is :  Kurian Benoy


## Competitions Journey

In [10]:
team_members_df = pd.read_csv("../input/meta-kaggle/TeamMemberships.csv")
team_df = pd.read_csv("../input/meta-kaggle/Teams.csv")
comp_df = pd.read_csv("../input/meta-kaggle/Competitions.csv")

temp_df = team_members_df[team_members_df["UserId"]==user_id]
temp_df = pd.merge(temp_df, team_df, left_on="TeamId", right_on="Id", how="left")
temp_df = pd.merge(temp_df, comp_df, left_on="CompetitionId", right_on="Id", how="left")
temp_df["DeadlineDate"] = pd.to_datetime(temp_df["DeadlineDate"], format="%m/%d/%Y %H:%M:%S")
temp_df["DeadlineYear"] = temp_df["DeadlineDate"].dt.year
temp_df["DeadlineDate"] = temp_df["DeadlineDate"].apply(lambda x: datetime.date(x.year,x.month,1))

temp_df = temp_df[~np.isnan(temp_df["PrivateLeaderboardRank"])]
temp_df.head()

,Id_x,TeamId,UserId,RequestDate,Id_y,CompetitionId,TeamLeaderId,TeamName,ScoreFirstSubmittedDate,LastSubmissionDate,PublicLeaderboardSubmissionId,PrivateLeaderboardSubmissionId,IsBenchmark,Medal,MedalAwardDate,PublicLeaderboardRank,PrivateLeaderboardRank,Id,Slug,Title,Subtitle,HostSegmentTitle,ForumId,OrganizationId,CompetitionTypeId,HostName,EnabledDate,DeadlineDate,ProhibitNewEntrantsDeadlineDate,TeamMergerDeadlineDate,TeamModelDeadlineDate,ModelSubmissionDeadlineDate,FinalLeaderboardHasBeenVerified,HasKernels,OnlyAllowKernelSubmissions,HasLeaderboard,LeaderboardPercentage,LeaderboardDisplayFormat,EvaluationAlgorithmAbbreviation,EvaluationAlgorithmName,EvaluationAlgorithmDescription,EvaluationAlgorithmIsMax,ValidationSetName,ValidationSetValue,MaxDailySubmissions,NumScoredSubmissions,MaxTeamSize,BanTeamMergers,EnableTeamModels,EnableSubmissionModelHashes,EnableSubmissionModelAttachments,RewardType,RewardQuantity,NumPrizes,UserRankMultiplier,CanQualifyTiers,TotalTeams,TotalCompetitors,TotalSubmissions,DeadlineYear
1,2527971,2474118,1310697,12/02/2018,2474118,6818,1310697.0,Making World great,NaN,12/06/2018,9399544.0,9399544.0,False,NaN,NaN,1163.0,1168.0,6818,humpback-whale-identification,Humpback Whale Identification,Can you identify a whale by its tail?,Featured,94474.0,4.0,1,HappyWhale,11/30/2018 20:37:03,2019-02-01,02/21/2019 23:59:00,02/21/2019 23:59:00,03/21/2019 00:00:00,NaN,True,True,False,True,20,0.0,MAP@{K},NaN,Mean Average Precision at K,True,Predicted Probability,"[0,1]",5,2,5.0,False,True,False,False,USD,25000.0,5,1.0,True,2129,2460,37522,2019
7,2527778,2473935,1310697,12/02/2018,2473935,10733,1310697.0,Santas,NaN,12/28/2018,9635610.0,9635610.0,False,NaN,NaN,570.0,570.0,10733,traveling-santa-2018-prime-paths,Traveling Santa 2018 - Prime Paths,"But does your code recall, the most efficient ...",Featured,89577.0,4.0,1,NaN,11/20/2018 17:58:08,2019-01-01,NaN,NaN,NaN,NaN,True,True,False,True,100,0.0,TravelingSanta2,Traveling Santa Metric 2 - Prime Edition,Returns TSP path length where every 10th step ...,False,NaN,NaN,10,2,5.0,False,False,False,False,USD,25000.0,3,1.0,True,1871,2046,21246,2019
14,3443216,3364681,1310697,07/03/2019,3364681,12716,1310697.0,Kurian Benoy,NaN,07/04/2019,11729303.0,11729303.0,False,NaN,NaN,163.0,164.0,12716,youtube8m-2019,The 3rd YouTube-8M Video Understanding Challenge,Temporal localization of topics within video,Research,260272.0,1623.0,1,NaN,06/27/2019 15:53:07,2019-10-01,10/04/2019 23:59:00,10/04/2019 23:59:00,10/25/2019 00:00:00,NaN,True,True,False,True,20,0.0,YT8M_MeanAveragePrecisionAtK,YT8M_MAP@{K},YT8M Custom MAP@{K} to handle partial ground t...,True,NaN,NaN,5,2,5.0,False,True,False,False,USD,25000.0,10,1.0,True,283,341,3753,2019
15,3994526,3898426,1310697,10/31/2019,3898426,12863,1310697.0,Kurian Benoy,NaN,11/11/2019,13227484.0,13227484.0,False,NaN,NaN,913.0,882.0,12863,tensorflow2-question-answering,TensorFlow 2.0 Question Answering,Identify the answers to real user questions ab...,Featured,412305.0,1099.0,1,NaN,10/28/2019 19:23:01,2020-01-01,01/15/2020 23:59:00,01/15/2020 23:59:00,02/15/2020 00:00:00,NaN,True,True,True,True,12,0.0,NQMicroF1,NQMicroF1,NQ Micro F1,True,NaN,NaN,5,2,5.0,False,True,False,False,USD,50000.0,3,1.0,True,1233,1417,8860,2020
17,3122054,3052118,1310697,04/16/2019,3052118,13435,1310697.0,thoriumkurian,NaN,04/22/2019,10964516.0,10964516.0,False,NaN,NaN,45.0,45.0,13435,aerial-cactus-identification,Aerial Cactus Identification,Determine whether an image contains a columnar...,Playground,144645.0,4.0,1,NaN,03/08/2019 19:22:51,2019-07-01,NaN,NaN,NaN,NaN,True,True,True,True,100,0.0,AUC,Area Under Receiver Operating Characteristic C...,Measures discrimination. Calculates how well a...,True,NaN,NaN,5,2,5.0,False,False,False,False,Knowledge,NaN,0,0.0,False,1225,1316,6781,2019


In [11]:
def scatter_plot(cnt_srs, color):
    trace = go.Scatter(
        x=cnt_srs.index[::-1],
        y=cnt_srs.values[::-1],
        showlegend=False,
        marker=dict(
            color=color,
        ),
    )
    return trace

cnt_df = temp_df.groupby('DeadlineYear')['PrivateLeaderboardRank'].agg(["size", "mean", "min"])
cnt_srs = cnt_df["size"]
cnt_srs = cnt_srs.sort_index()
trace = go.Bar(
    x=cnt_srs.index,
    y=cnt_srs.values,
    marker=dict(
        color="green",
    ),
)

layout = go.Layout(
    title='Count of competitions over years',
    font=dict(size=14),
    width=800,
    height=500,
)

data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="TargetCount")


### Mean Private Rank ###
cnt_srs = cnt_df["mean"]
cnt_srs = cnt_srs.sort_index()
trace = go.Bar(
    x=cnt_srs.index,
    y=cnt_srs.values,
    marker=dict(
        color="blue",
    ),
)

layout = go.Layout(
    title='Mean Rank over years',
    font=dict(size=14),
    width=800,
    height=500,
)

data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="TargetCount")



### Best rank each year ###
cnt_srs = cnt_df["min"]
cnt_srs = cnt_srs.sort_index()
trace = go.Bar(
    x=cnt_srs.index,
    y=cnt_srs.values,
    marker=dict(
        color="red",
    ),
)

layout = go.Layout(
    title='Best Rank in each year',
    font=dict(size=14),
    width=800,
    height=500,
)

data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="TargetCount")

In [12]:
#temp_df["Medal"].value_counts()

cnt_df = temp_df.pivot_table(index="DeadlineYear", columns="Medal", values="PrivateLeaderboardRank", aggfunc="count")
cnt_df = cnt_df.fillna(0)

def get_bar_chart(cnt_srs, name, color):
    trace = go.Bar(
        x=cnt_srs.index,
        y=cnt_srs.values,
        name=name,
        marker=dict(
            color=color,
        ),
    )
    return trace

medal_map = {1.:"Gold", 2.:"Silver", 3.:"Bronze"}
color_map = {1.:"gold", 2.:"silver", 3.:"darkorange"}
traces = []
for col in np.array(cnt_df.columns)[::-1]:
    cnt_srs = cnt_df[col]
    traces.append(get_bar_chart(cnt_srs, medal_map[col], color_map[col]))

layout = go.Layout(
    title='Competition Medals in each year',
    font=dict(size=14),
    barmode='stack',
    width=800,
    height=500,
)

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='stacked-bar')


In [13]:
cnt_srs = temp_df["HostSegmentTitle"].value_counts()

labels = (np.array(cnt_srs.index))
sizes = (np.array((cnt_srs / cnt_srs.sum())*100))

trace = go.Pie(labels=labels, values=sizes)
layout = go.Layout(
    title='Competition Type Distribution',
    font=dict(size=14),
    width=600,
    height=600,
)
data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="usertype")

In [14]:
comp_tags_df = pd.read_csv("../input/meta-kaggle/CompetitionTags.csv")
tags_df = pd.read_csv("../input/meta-kaggle/Tags.csv")
cnt_df = pd.merge(temp_df[["CompetitionId"]], comp_tags_df, on="CompetitionId", how="inner")
cnt_df = pd.merge(cnt_df, tags_df, left_on="TagId", right_on="Id", how="inner")
cnt_df["Name"].value_counts()

def bar_chart(cnt_srs, color):
    trace = go.Bar(
        x=cnt_srs.index,
        y=cnt_srs.values,
        showlegend=False,
        marker=dict(
            color=color,
        ),
    )
    return trace

cnt_srs = cnt_df["Name"].value_counts().head(10)
traces = [bar_chart(cnt_srs, "orange")]
layout = go.Layout(
    title='Data type of competitions',
    font=dict(size=14),
    barmode='stack',
    width=800,
    height=500,
)

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='stacked-bar')


In [15]:
cnt_srs = temp_df["TeamName"].value_counts().head(5)
traces = [bar_chart(cnt_srs, "blue")]
layout = go.Layout(
    title='Favorite Team Name',
    font=dict(size=14),
    barmode='stack',
    width=800,
    height=500,
)

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='stacked-bar')

In [16]:
temp_df = team_members_df[team_members_df["UserId"]==user_id]
temp_df = pd.merge(team_members_df, temp_df, on="TeamId", how="inner", suffixes=('', '_y'))
temp_df = temp_df[temp_df["UserId"]!=user_id]
temp_df = pd.merge(temp_df, users_df, left_on="UserId", right_on="Id", how="left")

cnt_srs = temp_df["DisplayName"].value_counts().head(7)
traces = [bar_chart(cnt_srs, "green")]
layout = go.Layout(
    title='Number of competitions with favorite team members ',
    font=dict(size=14),
    barmode='stack',
    width=800,
    height=500,
)

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='stacked-bar')

## Kernels Journey

In [17]:
kernels_df = pd.read_csv("../input/meta-kaggle/Kernels.csv")
temp_df = kernels_df[kernels_df["AuthorUserId"]==user_id]
temp_df["MadePublicDate"] = pd.to_datetime(temp_df["MadePublicDate"], format="%m/%d/%Y")
temp_df["MadePublicYear"] = temp_df["MadePublicDate"].dt.year
temp_df.head()

,Id,AuthorUserId,CurrentKernelVersionId,ForkParentKernelVersionId,ForumTopicId,FirstKernelVersionId,CreationDate,EvaluationDate,MadePublicDate,IsProjectLanguageTemplate,CurrentUrlSlug,Medal,MedalAwardDate,TotalViews,TotalComments,TotalVotes,MadePublicYear
231209,3601634,1310697,13269037.0,NaN,90348.0,12982877.0,04/16/2019 10:22:14,04/23/2019,2019-04-22,False,solving-aerial-cactus-challenge-using-fast-ai,3.0,06/04/2019,992,2,9,2019
245201,4389276,1310697,21734220.0,NaN,99812.0,15945161.0,06/19/2019 11:19:44,10/09/2019,2019-07-14,False,introduction-to-data-version-control-dvc,3.0,07/15/2019,3021,5,12,2019
245505,4412549,1310697,16018797.0,NaN,NaN,16012903.0,06/20/2019 13:13:22,06/20/2019,2019-06-20,False,cifar-kaggle,NaN,NaN,74,0,0,2019
248704,4576340,1310697,30393105.0,NaN,NaN,16483199.0,06/29/2019 15:04:04,03/18/2020,2019-06-29,False,image-classification-fastai-caltech-256,NaN,NaN,632,0,4,2019
250661,4698397,1310697,16843200.0,NaN,160834.0,16839342.0,07/07/2019 06:03:17,07/07/2019,2019-07-07,False,starter-kernel-fastai-discriminative-learning,3.0,09/20/2019,229,1,7,2019


In [18]:
# Number of kernels
cnt_srs = temp_df["MadePublicYear"].value_counts()
traces = [bar_chart(cnt_srs, "blue")]
layout = go.Layout(
    title='Number of kernels in each year',
    font=dict(size=14),
    width=800,
    height=500,
)

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='stacked-bar')


# Number of views
cnt_srs = temp_df.groupby("MadePublicYear")["TotalViews"].mean()
traces = [bar_chart(cnt_srs, "green")]
layout = go.Layout(
    title='Mean number of views per kernel',
    font=dict(size=14),
    width=800,
    height=500,
)

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='stacked-bar')

# Number of votes
cnt_srs = temp_df.groupby("MadePublicYear")["TotalVotes"].mean()
traces = [bar_chart(cnt_srs, "red")]
layout = go.Layout(
    title='Mean number of votes per kernel',
    font=dict(size=14),
    width=800,
    height=500,
)

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='stacked-bar')

In [19]:
cnt_df = temp_df.pivot_table(index="MadePublicYear", columns="Medal", values="AuthorUserId", aggfunc="count")
cnt_df = cnt_df.fillna(0)

def get_bar_chart(cnt_srs, name, color):
    trace = go.Bar(
        x=cnt_srs.index,
        y=cnt_srs.values,
        name=name,
        marker=dict(
            color=color,
        ),
    )
    return trace

medal_map = {1.:"Gold", 2.:"Silver", 3.:"Bronze"}
color_map = {1.:"gold", 2.:"silver", 3.:"darkorange"}
traces = []
for col in np.array(cnt_df.columns)[::-1]:
    cnt_srs = cnt_df[col]
    traces.append(get_bar_chart(cnt_srs, medal_map[col], color_map[col]))

layout = go.Layout(
    title='Kernel Medals in each year',
    font=dict(size=14),
    barmode='stack',
    width=800,
    height=500,
)

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='stacked-bar')

In [20]:
temp_df = kernels_df[kernels_df["AuthorUserId"]==user_id]
kernel_tags_df = pd.read_csv("../input/meta-kaggle/KernelTags.csv")
temp_df = pd.merge(temp_df, kernel_tags_df, left_on="Id", right_on="KernelId", how="inner")
temp_df = pd.merge(temp_df, tags_df, left_on="TagId", right_on="Id", how="inner")

cnt_srs = temp_df["Name"].value_counts().head(10)
traces = [bar_chart(cnt_srs, "orange")]
layout = go.Layout(
    title='Tag count of the kernels',
    font=dict(size=14),
    barmode='stack',
    width=800,
    height=500,
)

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='stacked-bar')

## Discussion Journey

In [ ]:
#temp_df = kernels_df[kernels_df["AuthorUserId"]==user_id]
forum_message_df = pd.read_csv("../input/meta-kaggle/ForumMessages.csv")
temp_df = forum_message_df[forum_message_df["PostUserId"]==user_id]
temp_df["PostDate"] = pd.to_datetime(temp_df["PostDate"], format="%m/%d/%Y %H:%M:%S")
temp_df["PostYear"] = temp_df["PostDate"].dt.year
temp_df.head()

In [ ]:
# Number of kernels
cnt_srs = temp_df["PostYear"].value_counts()
traces = [bar_chart(cnt_srs, "blue")]
layout = go.Layout(
    title='Number of forum posts in each year',
    font=dict(size=14),
    width=800,
    height=500,
)

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='stacked-bar')

In [ ]:
cnt_df = temp_df.pivot_table(index="PostYear", columns="Medal", values="PostUserId", aggfunc="count")
cnt_df = cnt_df.fillna(0)

def get_bar_chart(cnt_srs, name, color):
    trace = go.Bar(
        x=cnt_srs.index,
        y=cnt_srs.values,
        name=name,
        marker=dict(
            color=color,
        ),
    )
    return trace

medal_map = {1.:"Gold", 2.:"Silver", 3.:"Bronze"}
color_map = {1.:"gold", 2.:"silver", 3.:"darkorange"}
traces = []
for col in np.array(cnt_df.columns)[::-1]:
    cnt_srs = cnt_df[col]
    traces.append(get_bar_chart(cnt_srs, medal_map[col], color_map[col]))

layout = go.Layout(
    title='Discussion Medals in each year',
    font=dict(size=14),
    barmode='stack',
    width=800,
    height=500,
)

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='stacked-bar')

In [8]:
import re
def clean_string(txt):
    txt = str(txt)
    txt = re.sub("<.*?>", "", txt)
    txt = re.sub(' +', ' ', txt)
    return txt

temp_df["Message"] = temp_df["Message"].apply(lambda x: clean_string(x))
#temp_df.head()

from wordcloud import WordCloud, STOPWORDS

# Thanks : https://www.kaggle.com/aashita/word-clouds-of-various-shapes ##
def plot_wordcloud(text, mask=None, max_words=200, max_font_size=100, figure_size=(24.0,16.0), 
                   title = None, title_size=40, image_color=False):
    stopwords = set(STOPWORDS)
    more_stopwords = {'one', 'br', 'Po', 'th', 'sayi', 'fo', 'Unknown'}
    stopwords = stopwords.union(more_stopwords)

    wordcloud = WordCloud(background_color='black',
                    stopwords = stopwords,
                    max_words = max_words,
                    max_font_size = max_font_size, 
                    random_state = 42,
                    width=800, 
                    height=400,
                    mask = mask)
    wordcloud.generate(str(text))
    
    plt.figure(figsize=figure_size)
    if image_color:
        image_colors = ImageColorGenerator(mask);
        plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear");
        plt.title(title, fontdict={'size': title_size,  
                                  'verticalalignment': 'bottom'})
    else:
        plt.imshow(wordcloud);
        plt.title(title, fontdict={'size': title_size, 'color': 'black', 
                                  'verticalalignment': 'bottom'})
    plt.axis('off');
    plt.tight_layout()  
    
plot_wordcloud(temp_df["Message"], title="Word Cloud of the common words in forum messages")

NameError: name 'temp_df' is not defined

### More to come. Stay tuned!

Some of the ideas are
* Kagglers who upvote your kernels
* Kagglers who upvote your forum posts
* Number of followers over time